In [25]:
def combine(command, parameter, reuse = None):
    if(reuse == None):
        reuse = {}
    pointer = 0
    line = ''
    changed = False
    while(pointer < len(command)):
        count = 0
        no_space_start = pointer-1
        while(pointer < len(command)):
            if command[pointer] == '{':
                count += 1
            else:
                if(command[pointer] == ' ' or command[pointer] == '\t'):
                    line += command[pointer]
                else:
                    line = ''
                    no_space_start = pointer
                count = 0
            pointer += 1      
            if(count == 2):
                start = pointer
                break
            
        count = 0
        while(pointer < len(command)):
            if command[pointer] == '}':
                count += 1
            else:
                count = 0
            pointer += 1                
            if(count == 2):
                current = command[start:pointer-2].strip()
                if not current in reuse.keys() or reuse[current] == 0:
                    reuse[current] = 1
                else:
                    reuse[current] += 1
                    current = current + ' ' + str(reuse[current])
                if(current in parameter.keys()):
                    value = parameter[current]
                    endn = value[-1] == '\n'
                    value = value.split('\n')
                    value = [i for i in value if i != '']
                    if len(value) > 1:
                        tv = ''
                        for v in value:
                            tv += line + v + '\n'
                    else:
                        tv = line + value[0] + ('\n' if endn else '')
                    changed = True
                    command = command[:no_space_start+1] + tv + command[pointer:]
                    pointer = no_space_start + len(parameter[current])-1
                else:
                    reuse[current] = 0
                    command = command[:start] + ' ' + current + ' }}' + command[pointer:]
                break
    if(changed):
        command = combine(command, parameter, reuse)
    return command

code = '''{{ command }} {{ head }}:
    {{ body }}
'''

main = '''{{ body }}'''

foreach = combine(code,{
    'command': 'for',
    'head': '{{ pointer }} in {{ literable }}',
})

function_def = combine(code,{
    'command': 'def',
    'head': '{{ name }}({{ parameter }})'
})

function_call = '{{ name }}({{ parameter }})'
variable_def = '{{ name }} = {{ value }}'

write = combine(function_call,{
    'name':'print',
    'parameter':'{{ text }}'
})

_import = 'import {{ module }}\n'
from_import = 'from {{ module }} import {{ component }}\n'
from_import_all = combine(from_import,{
    'component': '*'
})

In [27]:
pointer_1 = 'i'
pointer_2 = 'j'
parameter = 'array'

inner_loop = combine(foreach, {''
    'literable': pointer_1,
    'pointer': pointer_2,
    'body': write,
    'text': pointer_2
})

outer_loop = combine(foreach,{
    'literable': parameter,
    'pointer': pointer_1,
    'body': inner_loop,
})

main_body = combine(function_def,{
    'name': 'bar',
    'parameter':parameter,
    'body': outer_loop
})

print(main_body)

def bar(array):
    for i in array:
        for j in i:
            print(j)




In [8]:
pointer_1 = 'i'
pointer_2 = 'j'
parameter = 'array'

inner_loop = combine(foreach, {''
    'literable': pointer_1,
    'pointer': pointer_2,
    'body': write,
    'text': pointer_2
})

outer_loop = combine(foreach,{
    'literable': parameter,
    'pointer': pointer_1,
    'body': inner_loop,
})

main_body = combine(function_def,{
    'name': 'bar',
    'parameter':parameter,
    'body': outer_loop
})

print(inner_loop)

for {{ pointer }} in {{ literable }}:
    {{ body }}



In [7]:

imports = combine(main,
{
    'body': from_import_all * 5 +_import,
    'module'  : 'pygame.locals',
    'module 2': 'OpenGL.GL',
    'module 3': 'OpenGL.GLU',
    'module 4': 'OpenGL.GLUT',
    'module 5': 'geo_math',
    'module 6': 'pygame',

},{})
print(imports)

from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
from geo_math import *
import pygame

